In [1]:
# install the open cv library
!pip install opencv-python

In [16]:
# importing the packages
import pandas as pd
import os
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.patches as mpatches
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import precision_score, recall_score, f1_score
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [3]:
# there are approx 5 features present overall across 75000 images
features_dataframe = pd.read_csv("final_dataset_containing_5_extracted_features.csv")

In [4]:
features_dataframe.shape

(72986, 12)

In [5]:
# viewing the features_dataframe
features_dataframe.head()

,file_name,feature_x_1,feature_x_2,feature_x_3,feature_x_4,feature_x_5,feature_y_1,feature_y_2,feature_y_3,feature_y_4,feature_y_5,is_gan
0,00000.png,125,101.0,73.0,NaN,NaN,89,130.0,87.0,NaN,NaN,No
1,000004.png,127,75.0,NaN,NaN,NaN,90,87.0,NaN,NaN,NaN,Yes
2,00001.png,126,75.0,76.0,NaN,NaN,91,90.0,139.0,NaN,NaN,No
3,000014.png,130,77.0,NaN,NaN,NaN,89,91.0,NaN,NaN,NaN,Yes
4,000016.png,127,173.0,74.0,NaN,NaN,90,61.0,91.0,NaN,NaN,Yes


In [6]:
# performing shuffling on features_dataframe
features_dataframe = features_dataframe.sample(frac=1)
features_dataframe.head()

,file_name,feature_x_1,feature_x_2,feature_x_3,feature_x_4,feature_x_5,feature_y_1,feature_y_2,feature_y_3,feature_y_4,feature_y_5,is_gan
44842,24368.png,126,74.0,NaN,NaN,NaN,90,89.0,NaN,NaN,NaN,No
14212,042796.png,124,105.0,79.0,87.0,107.0,87,121.0,89.0,121.0,69.0,Yes
71235,52150.png,82,120.0,NaN,NaN,NaN,136,137.0,NaN,NaN,NaN,No
46712,26360.png,76,100.0,124.0,NaN,NaN,89,132.0,88.0,NaN,NaN,No
54421,34466.png,127,78.0,NaN,NaN,NaN,89,87.0,NaN,NaN,NaN,No


In [7]:
features_dataframe = features_dataframe.fillna(-1)

In [8]:
# normalizing the data
is_gan = features_dataframe["is_gan"] 
features_dataframe.drop(columns=["file_name", "is_gan"], inplace=True)

In [9]:
standard_scalar = StandardScaler()
features_dataframe = standard_scalar.fit_transform(features_dataframe)
features_dataframe = pd.DataFrame(features_dataframe)
features_dataframe

,0,1,2,3,4,5,6,7,8,9
0,0.751776,-0.326736,-0.923653,-0.432465,-0.189726,-0.188229,0.127754,-0.899031,-0.420531,-0.183362
1,0.670618,0.486934,0.664967,1.912526,5.609828,-0.370958,1.045013,0.574783,2.315059,3.142019
2,-1.033702,0.880646,-0.923653,-0.432465,-0.189726,2.613612,1.503643,-0.899031,-0.420531,-0.183362
3,-1.277176,0.355697,1.558566,-0.432465,-0.189726,-0.249139,1.360321,0.558408,-0.420531,-0.183362
4,0.792355,-0.221746,-0.923653,-0.432465,-0.189726,-0.249139,0.070425,-0.899031,-0.420531,-0.183362
...,...,...,...,...,...,...,...,...,...,...
72981,0.630039,-0.116757,1.062123,-0.432465,-0.189726,-0.370958,0.070425,1.426320,-0.420531,-0.183362
72982,-1.236597,1.011883,-0.923653,-0.432465,-0.189726,-0.249139,0.099089,-0.899031,-0.420531,-0.183362
72983,0.832934,-0.274241,0.605394,-0.432465,-0.189726,-0.310048,0.156418,1.360817,-0.420531,-0.183362
72984,0.832934,-0.274241,-0.923653,-0.432465,-0.189726,-0.310048,0.099089,-0.899031,-0.420531,-0.183362


In [10]:
features_dataframe["is_gan"] = is_gan
features_dataframe["is_gan"] = features_dataframe["is_gan"].map(dict(Yes=1, No=0))
features_dataframe

,0,1,2,3,4,5,6,7,8,9,is_gan
0,0.751776,-0.326736,-0.923653,-0.432465,-0.189726,-0.188229,0.127754,-0.899031,-0.420531,-0.183362,0
1,0.670618,0.486934,0.664967,1.912526,5.609828,-0.370958,1.045013,0.574783,2.315059,3.142019,1
2,-1.033702,0.880646,-0.923653,-0.432465,-0.189726,2.613612,1.503643,-0.899031,-0.420531,-0.183362,0
3,-1.277176,0.355697,1.558566,-0.432465,-0.189726,-0.249139,1.360321,0.558408,-0.420531,-0.183362,1
4,0.792355,-0.221746,-0.923653,-0.432465,-0.189726,-0.249139,0.070425,-0.899031,-0.420531,-0.183362,1
...,...,...,...,...,...,...,...,...,...,...,...
72981,0.630039,-0.116757,1.062123,-0.432465,-0.189726,-0.370958,0.070425,1.426320,-0.420531,-0.183362,0
72982,-1.236597,1.011883,-0.923653,-0.432465,-0.189726,-0.249139,0.099089,-0.899031,-0.420531,-0.183362,0
72983,0.832934,-0.274241,0.605394,-0.432465,-0.189726,-0.310048,0.156418,1.360817,-0.420531,-0.183362,0
72984,0.832934,-0.274241,-0.923653,-0.432465,-0.189726,-0.310048,0.099089,-0.899031,-0.420531,-0.183362,0


In [11]:
features_dataframe.isna().sum()

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
is_gan    0
dtype: int64

In [12]:
x = features_dataframe
y = features_dataframe["is_gan"]
x.drop(columns=["is_gan"], inplace=True)

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

In [14]:
f"Shape of training data = {x_train.shape[0]} and testing data = {x_test.shape[0]}"

'Shape of training data = 51090 and testing data = 21896'

## Performing experiments using the following hyperparameters to find the best model

In [17]:
# Performing Kfold cross validation
knn_pipeline = Pipeline([("classifier", GaussianNB())])
param_grid = {}

knn_pipeline

Pipeline(steps=[('classifier', GaussianNB())])

In [18]:
gridsearch_cv = GridSearchCV(knn_pipeline, param_grid=param_grid, cv=10, verbose=2)

In [19]:
gridsearch_cv.fit(x_train, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


GridSearchCV(cv=10, estimator=Pipeline(steps=[('classifier', GaussianNB())]),
             param_grid={}, verbose=2)

In [20]:
gridsearch_cv.best_params_

{}

## Training the model on the above best parameters

In [21]:
nb_algorithm = GaussianNB()
nb_algorithm.fit(x_train, y_train)

GaussianNB()

In [22]:
nb_predictions = nb_algorithm.predict(x_test)

In [23]:
cm = metrics.confusion_matrix(y_test, nb_predictions)
print(f"The confusion matrix for Logistic Regression is given below: \n {cm}")

The confusion matrix for Logistic Regression is given below: 
 [[14908     0]
 [ 6988     0]]


In [25]:
nb_precision_score = precision_score(y_test, nb_predictions, average='weighted', labels=np.unique(nb_predictions))
nb_precision_score

0.6808549506759225

In [26]:
nb_recall = recall_score(y_test, nb_predictions, average='weighted', labels=np.unique(nb_predictions))
nb_recall

1.0

In [27]:
nb_f1 = f1_score(y_test, nb_predictions, average='weighted', labels=np.unique(nb_predictions))
nb_f1

0.8101293337680686

In [28]:
# storing the svm model
filename = 'naive_bayes_final.pkl'
pickle.dump(nb_algorithm, open(filename, 'wb'))